In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [8]:
!kaggle datasets download -d sorawitsinlapanurak/top-10-best-rated-thai-food

 96% 193M/201M [00:01<00:00, 95.1MB/s]
100% 201M/201M [00:02<00:00, 104MB/s] 


In [9]:
!unzip /content/top-10-best-rated-thai-food.zip

Archive:  /content/top-10-best-rated-thai-food.zip
  inflating: top10_thai_food_dataset/test/Green_Curry/004.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/013.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/014.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/016.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/017.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/045.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/048.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/053.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/058.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/072.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/080.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/082.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/102.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry/111.jpg  
  inflating: top10_thai_food_dataset/test/Green_Curry

Load Data

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

In [36]:
train_dir = '/content/top10_thai_food_dataset/train'
validation_dir = '/content/top10_thai_food_dataset/val'
test_dir = '/content/top10_thai_food_dataset/test'

In [37]:
import os

In [38]:
kategori = os.listdir(path)
kategori

['train', 'test', 'val']

Menggunakan ImageDataGenerator untuk  melakukan augmentasi pada data

In [39]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [40]:
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train, val, dan test

In [41]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 1977 images belonging to 10 classes.


In [42]:
vvalidation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 564 images belonging to 10 classes.


In [43]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 285 images belonging to 10 classes.


Membuat model sequential


In [44]:
model = Sequential()


# Menggunakan pre-trained model VGG16

In [45]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
base_model.trainable = False

model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

58889256/58889256 [==============================] - 0s 0us/step


In [46]:
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

Callbacks

In [47]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3)

## Latih Model

In [51]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

history = model.fit(
    train_generator,
    epochs=3,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)


Found 564 images belonging to 10 classes.
Epoch 1/3
62/62 [==============================] - 647s 10s/step - loss: -67625.0938 - accuracy: 0.1062 - val_loss: -87264.6094 - val_accuracy: 0.1064 - lr: 0.0010
Epoch 2/3
62/62 [==============================] - 702s 11s/step - loss: -111747.7812 - accuracy: 0.1062 - val_loss: -136908.3438 - val_accuracy: 0.1064 - lr: 0.0010
Epoch 3/3
62/62 [==============================] - 668s 11s/step - loss: -167171.1406 - accuracy: 0.1062 - val_loss: -197346.9375 - val_accuracy: 0.1064 - lr: 0.0010


Evaluasi model

In [52]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Akurasi pada data test: {test_accuracy:.2f}')

9/9 [==============================] - 72s 8s/step - loss: -198931.5312 - accuracy: 0.1053
Akurasi pada data test: 0.11


Membuat grafik pergerakan metrik Accuracy dan Loss terhadap epoch pada Training dan Validation

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

Grafik Accuracy

In [ ]:
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'r', label='Train')
plt.plot(epochs, val_acc, 'b', label='Val')
plt.title('Model Accuracy')
plt.legend()

Grafik Loss

In [ ]:
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'r', label='Train')
plt.plot(epochs, val_loss, 'b', label='Val')
plt.title('Model Loss')
plt.legend()

plt.show()

link Kaggle
